In [1]:

!pip install -U bitsandbytes

In [2]:
import gc
import torch
import bitsandbytes
gc.collect()


38

In [3]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [4]:
import transformers
import accelerate
import tokenizers

In [5]:
# print(gc.__version__)
print(torch.__version__)
print(transformers.__version__)
print(bitsandbytes.__version__)
print(transformers.__version__)
print(accelerate.__version__)
print(tokenizers.__version__)

2.5.1+cu121
4.47.0
0.48.2
4.47.0
1.2.1
0.21.0


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import os


In [7]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [8]:


model_name = "/kaggle/input/deepseek-v2/transformers/deepseek-v2-lite-chat/1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
config = AutoConfig.from_pretrained(model_name,trust_remote_code=True)

In [9]:
config.max_position_embeddings = 512  
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  
    llm_int8_threshold=3.0,)

In [10]:



model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,  
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto", 
)

model.gradient_checkpointing_disable()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
print("Device Map:", model.hf_device_map)
print("Is CUDA available?", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("GPU Memory Usage (GB):", torch.cuda.memory_allocated(0) / 1e9)

Device Map: {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.norm': 1, 'lm_head': 1}
Is CUDA available? True
Number of GPUs: 2
GPU Memory Usage (GB): 6.595700736


In [12]:
user_prompt = "What is the area of a rectangle with a length of 8 cm and a width of 5 cm?"
inputs = tokenizer(user_prompt, return_tensors="pt", truncation=True).to("cuda:0")  
outputs = model.generate(**inputs, max_new_tokens=100)
model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_output)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


What is the area of a rectangle with a length of 8 cm and a width of 5 cm?

To find the area of a rectangle, we use the formula:

Area = Length × Width

In this case, the length is 8 cm and the width is 5 cm. So, we have:

Area = 8 cm × 5 cm

Area = 40 square centimeters

The area of the rectangle is 40 square centimeters.


In [13]:
user_prompt = "solve this equation: 3(x-8)=2(x+1)+10"
inputs = tokenizer(user_prompt, return_tensors="pt", truncation=True).to("cuda:0")  
outputs = model.generate(**inputs, max_new_tokens=100)
model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_output)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


solve this equation: 3(x-8)=2(x+1)+10
3(x-8)=2(x+1)+10
Step 1: Distribute the 3 and 2 on the left and right sides of the equation, respectively.
3(x-8) = 3x - 24
2(x+1) = 2x + 2
Now the equation is:
3x - 24 = 2x + 2 + 10
Step 2: Subtract


In [14]:
user_prompt = "Every matrix A satisfying A2 = A is similar to a diagonal matrix."
inputs = tokenizer(user_prompt, return_tensors="pt",truncation=True).to("cuda:0")  
outputs = model.generate(**inputs, max_new_tokens=1000) 
model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_output)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Every matrix A satisfying A2 = A is similar to a diagonal matrix.
Proof.
Let B = A - I, where I is the identity matrix. Then B2 = B, so B is nilpotent. By the Jordan canonical form theorem, there exists a basis of Cn such that B is upper triangular. Then A is similar to a diagonal matrix.
QED  
Corollary.
Every matrix A satisfying A2 = 0 is similar to a diagonal matrix.
Proof.
If A2 = 0, then A is nilpotent, so by the Jordan canonical form theorem, A is similar to a diagonal matrix.
QED


In [15]:
user_prompt = "Let A be a 3×3 matrix with real entries. Prove that if A is not similar over R to a triangular matrix then A is similar over C to a diagonal matrix."
inputs = tokenizer(user_prompt, return_tensors="pt", truncation=True).to("cuda:0")  
outputs = model.generate(**inputs, max_new_tokens=1000) 
model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(model_output)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Let A be a 3×3 matrix with real entries. Prove that if A is not similar over R to a triangular matrix then A is similar over C to a diagonal matrix.

Here's what I have so far:

Assume A is not similar over R to a triangular matrix. Then there exists a matrix B such that B and A are not similar over R. Since A and B are similar over R, they have the same minimal polynomial over R. Let m_A(x) and m_B(x) be the minimal polynomials of A and B, respectively.

Since A and B are similar over R, they have the same eigenvalues with the same algebraic and geometric multiplicities. Let λ_1, λ_2, ..., λ_n be the eigenvalues of A, and let μ_1, μ_2, ..., μ_n be the eigenvalues of B.

Since A and B have the same minimal polynomial, m_A(x) = m_B(x). Let m_A(x) = m_B(x) = (x - λ_1)^e_1 * (x - λ_2)^e_2 * ... * (x - λ_n)^e_n, where e_i are the algebraic multiplicities of λ_i.

Now, consider the minimal polynomial m_C(x) of A over C. Since A and B have the same eigenvalues with the same algebraic and geo